In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.cluster import OPTICS, cluster_optics_dbscan

/home/ivo/Programming_Personal_Projects/cluster_proj/clustering-problem/.ct_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ivo/Programming_Personal_Projects/cluster_proj/clustering-problem/.ct_env/lib/python3.10/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L3-v2')


In [3]:
data = pd.read_csv("../data/location_data.csv",index_col=None)

In [4]:
data.describe()

,city,road,house_number,postcode,state
count,63695,63695,63695,63695,63695
unique,5534,17184,6353,11015,121
top,houston,main st,101,46227,ca
freq,640,599,357,33,3251


In [5]:
data.head()

,city,road,house_number,postcode,state
0,fort worth,sycamore school rd,5421,76123,texas
1,gautier,highway 90,2701,39553,mississippi
2,farmington,e main,4750,87401,nm
3,richland,queensgate drive,2751,99352,washington
4,davie,s state hwy seven,1600,33317,fl


In [6]:
data.shape

(63695, 5)

In [7]:
data_no_dup = data.drop_duplicates(ignore_index=True)
data_no_dup.shape # we have many duplicate rows

(41423, 5)

In [8]:
data_no_dup

,city,road,house_number,postcode,state
0,fort worth,sycamore school rd,5421,76123,texas
1,gautier,highway 90,2701,39553,mississippi
2,farmington,e main,4750,87401,nm
3,richland,queensgate drive,2751,99352,washington
4,davie,s state hwy seven,1600,33317,fl
...,...,...,...,...,...
41418,highland,baseline st,27774,92346,california
41419,hilliard,main st,3685,43026,ohio
41420,window rock,window rock loop road,264,86515,az
41421,denver,s santa fe dr,1395,80223,co


In [19]:
# sentences = data_no_dup["city"]+" "+ data_no_dup["road"] +" "+ data_no_dup["house_number"]\
#             +" "+ data_no_dup["postcode"]+" "+ data_no_dup["state"]
sentences = data_no_dup["city"]+"; "+ data_no_dup["state"]
# I had hoped that the model would group well based on state and city
# the full address would contain too much noise for the model, I think.

In [20]:
sentences[34357] # this doesn't seem to be an address in the USA...
# this will create extra difficulties, and we may need to use a multi-lingual model

'kota jkt utara; daerah khusus ibukota jakarta'

In [21]:
sentence_embeddings = model.encode(sentences)
sentence_embeddings = pd.DataFrame(sentence_embeddings)
# this will not scale well...


In [22]:
sentence_embeddings

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.152617,0.346525,0.106051,0.233081,0.245558,0.451261,0.017126,0.254467,-0.394009,-1.010338,...,-0.064418,-0.235545,0.283173,0.237124,0.186759,-0.646233,-0.218449,-0.735229,0.109549,0.156433
1,0.034952,-0.169938,-0.027002,0.057674,0.050665,0.083432,0.561005,0.109844,-0.377877,-0.292408,...,0.095087,-0.084000,0.030543,-0.278375,-0.063311,0.171059,-0.237557,-0.270517,-0.048645,0.004052
2,0.281359,-0.082562,0.173897,0.100577,0.020736,-0.214602,0.364283,-0.096042,-0.091129,-0.030768,...,-0.017165,-0.165390,-0.216237,-0.450608,0.469142,-0.187986,-0.140085,0.311051,-0.072850,-0.100385
3,0.373036,-0.367566,0.231408,0.089526,0.280310,-0.136095,0.154693,0.157684,-0.249141,-0.582114,...,0.057515,0.503825,-0.183302,0.366384,-0.210844,-0.257483,0.128817,-0.177661,-0.221699,0.134845
4,0.038945,-0.147470,0.398138,-0.320627,-0.192179,0.071997,0.651607,0.344741,0.190151,-0.322179,...,-0.487213,-0.262816,-0.061468,0.041563,-0.640033,-0.359333,-0.299477,-0.624583,0.686342,0.191711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41418,0.102373,-0.156166,0.268159,0.040496,-0.082995,0.188021,-0.236241,-0.392205,-0.724145,-0.170360,...,-0.145533,-0.426664,-0.236493,-0.403409,0.062207,-0.786934,0.632506,0.027985,-0.169292,-0.053576
41419,0.078716,0.329421,0.020217,0.010534,0.147073,0.093543,0.095245,-0.370114,0.054287,-0.510097,...,0.063167,0.162539,-0.346958,-0.033109,0.289650,-0.208304,0.182964,0.242674,0.064733,-0.642664
41420,-0.057861,0.010629,0.366493,0.603365,-0.251854,-0.072574,0.659337,-0.122034,0.056329,-0.620800,...,0.385658,-0.040526,-0.712246,-0.104852,0.064459,-0.141733,-0.302730,0.101295,-0.158719,0.011262
41421,-0.036455,-0.377972,-0.093594,-0.788493,0.478090,0.261365,-0.182887,-0.132093,0.117873,-0.135003,...,0.141695,0.244757,-0.103862,-0.351605,-0.307580,-0.154773,0.057898,0.100020,0.369203,-0.285767


In [23]:
sentence_embeddings.to_csv("../data/embeddings.csv")
#from operator import index


#sentence_embeddings = pd.read_csv("../data/embeddings.csv", index_col = 0)

In [24]:
clustering = OPTICS(min_samples=2,
                    n_jobs = -1,xi = 0.0001,p=30).fit(sentence_embeddings[:100])

/home/ivo/Programming_Personal_Projects/cluster_proj/clustering-problem/.ct_env/lib/python3.10/site-packages/sklearn/cluster/_optics.py:903: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


In [25]:
clustering.labels_

array([-1, -1, -1,  1,  7, -1, -1,  2, 16, 18, -1, -1, 10, -1,  6,  3, 11,
       -1, -1, -1, 18,  6,  7,  0, 18, 20,  1, 12, 10, -1,  4, -1, -1, -1,
       -1, -1,  3,  4,  6, -1,  9, -1,  5,  6, -1, -1,  1, 15, -1, -1, -1,
        3, -1, 13,  4,  0,  0, -1, -1, -1,  1, 11, 11,  3, 14,  1,  9, -1,
       19, 13,  4, 19, -1, -1, -1, 11, 20, -1, 15,  8, 12,  4,  8, 17,  2,
        0,  4,  5,  3, -1, 14, -1, -1, -1,  4, -1, 17,  7, 10, 16])

In [26]:
index = sentence_embeddings[:100][clustering.labels_==3].index

for id in index:
    print(sentences[:100][id])
# the results are not good...

otisville; michigan
phenix city; alabama
hollywood; ca
madisonville; tx
lubbock; tx


In [27]:
for id in index:
    print(id)

15
36
51
63
88
